In [61]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [62]:
import random
random.seed(5)

In [63]:
path = 'C:\\Users\\pereg\\AppData\\Local\\Temp\\pulsar\\amazon.dataset.csv'

# read data and apply one-hot encoding
df = pd.read_csv(path)
df.head()
# sns.countplot(x='Label', data=df)

X = df.iloc[0:, 1:].values
Y = df.iloc[0:, 0].values

In [64]:
X

array([[ 471.3, 1348.3,   66.7, ...,    0. ,    0. ,    0. ],
       [ 471.3, 1348.3,   66.7, ...,    0. ,    0. ,    0. ],
       [ 258.7,  636.7,  639. , ...,    0. ,    0. ,    0. ],
       ...,
       [ 439.3, 1098.5,  145.3, ...,    0. ,    0. ,    0. ],
       [ 439.3, 1098.5,  145.3, ...,    0. ,    0. ,    0. ],
       [ 439.3, 1098.5,  145.3, ...,    0. ,    0. ,    0. ]])

In [65]:
Y

array([4, 4, 5, 5, 6, 6, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5,
       5, 6, 6, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 4, 4, 5, 5,
       6, 6, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 4, 4, 5, 5, 6,
       6, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5, 5, 6, 6, 1, 1,
       2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 4, 4, 5, 5, 6, 6,
       1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0], dtype=int64)

In [66]:
from sklearn.model_selection import train_test_split
x, x_val, y, y_val = train_test_split(X, Y, test_size=0.33, random_state=42)

In [67]:
x

array([[ 407.3,  778.1,   72.2, ...,    0. ,    0. ,    0. ],
       [ 944.3, 1348.3,   89. , ...,    0. ,    0. ,    0. ],
       [ 439.3, 1005. ,   72.2, ...,    0. ,    0. ,    0. ],
       ...,
       [ 439.3, 1005. ,   72.2, ...,    0. ,    0. ,    0. ],
       [ 971.3, 1005. ,   72.2, ...,    0. ,    0. ,    0. ],
       [ 439.3, 1098.5,  145.3, ...,    0. ,    0. ,    0. ]])

In [68]:
x.shape, y.shape, x_val.shape, y_val.shape

((85, 276), (85,), (43, 276), (43,))

In [69]:
x.shape, y.shape

((85, 276), (85,))

In [70]:
x_train = x.reshape(-1, x.shape[1]).astype('float32')

In [71]:
y_train = y

x_val = x_val.reshape(-1, x_val.shape[1]).astype('float32')
y_val = y_val

In [72]:
x_train.shape

(85, 276)

In [73]:
x_val = torch.from_numpy(x_val)
y_val = torch.from_numpy(y_val)

In [74]:
from torch.utils.data import Dataset, DataLoader
class Data(Dataset):
    def __init__(self):
        self.x=torch.from_numpy(x).type(torch.FloatTensor)
        self.y=torch.from_numpy(y).type(torch.LongTensor)
        self.len=self.x.shape[0]
    def __getitem__(self,index):
        return self.x[index], self.y[index]
    def __len__(self):
        return self.len

In [75]:
data_set=Data()

In [76]:
trainloader=DataLoader(dataset=data_set,batch_size=10)

In [77]:
data_set.x[0:10]

tensor([[ 407.3000,  778.1000,   72.2000,  ...,    0.0000,    0.0000,
            0.0000],
        [ 944.3000, 1348.3000,   89.0000,  ...,    0.0000,    0.0000,
            0.0000],
        [ 439.3000, 1005.0000,   72.2000,  ...,    0.0000,    0.0000,
            0.0000],
        ...,
        [ 439.3000, 1005.0000,   72.2000,  ...,    0.0000,    0.0000,
            0.0000],
        [ 408.3000,  660.3000,   80.0000,  ...,    0.0000,    0.0000,
            0.0000],
        [ 439.3000, 1005.0000,   72.2000,  ...,    0.0000,    0.0000,
            0.0000]])

In [78]:
data_set.y[0:10]

tensor([3, 4, 3, 2, 3, 6, 1, 3, 2, 3])

In [79]:
data_set.x.shape, data_set.y.shape

(torch.Size([85, 276]), torch.Size([85]))

In [80]:
class Net(nn.Module):
    def __init__(self,in_size,n_hidden1,n_hidden2,out_size,p=0):

        super(Net,self).__init__()
        self.drop=nn.Dropout(p=p)
        self.linear1=nn.Linear(in_size,n_hidden1)
        nn.init.kaiming_uniform_(self.linear1.weight,nonlinearity='relu')
        self.linear2=nn.Linear(n_hidden1,n_hidden2)
        nn.init.kaiming_uniform_(self.linear1.weight,nonlinearity='relu')
        self.linear3=nn.Linear(n_hidden2,n_hidden2)
        nn.init.kaiming_uniform_(self.linear3.weight,nonlinearity='relu')
        self.linear4=nn.Linear(n_hidden2,out_size)

    def forward(self,x):
        x=F.relu(self.linear1(x))
        x=self.drop(x)
        x=F.relu(self.linear2(x))
        x=self.drop(x)
        x=F.relu(self.linear3(x))
        x=self.drop(x)
        x=self.linear4(x)
        return x

In [81]:
model=Net(276,50,30,7)
model_drop=Net(276,50,30,7,p=0.2)
model_drop

Net(
  (drop): Dropout(p=0.2, inplace=False)
  (linear1): Linear(in_features=276, out_features=50, bias=True)
  (linear2): Linear(in_features=50, out_features=30, bias=True)
  (linear3): Linear(in_features=30, out_features=30, bias=True)
  (linear4): Linear(in_features=30, out_features=7, bias=True)
)

In [82]:
model_drop.train()

Net(
  (drop): Dropout(p=0.2, inplace=False)
  (linear1): Linear(in_features=276, out_features=50, bias=True)
  (linear2): Linear(in_features=50, out_features=30, bias=True)
  (linear3): Linear(in_features=30, out_features=30, bias=True)
  (linear4): Linear(in_features=30, out_features=7, bias=True)
)

In [83]:
optimizer_ofit = torch.optim.Adam(model.parameters(), lr=0.01)
optimizer_drop = torch.optim.Adam(model_drop.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [84]:
LOSS={}
LOSS['training data no dropout']=[]
LOSS['validation data no dropout']=[]
LOSS['training data dropout']=[]
LOSS['validation data dropout']=[]

In [85]:
n_epochs=200

for epoch in range(n_epochs):
    for x, y in trainloader:
        #make a prediction for both models
        yhat = model(data_set.x)
        yhat_drop = model_drop(data_set.x)
        #calculate the lossf or both models
        loss = criterion(yhat, data_set.y)
        loss_drop = criterion(yhat_drop, data_set.y)

        #store the loss for  both the training and validation  data for both models
        LOSS['training data no dropout'].append(loss.item())
        LOSS['training data dropout'].append(loss_drop.item())
        model_drop.eval()
        model_drop.train()

        #clear gradient
        optimizer_ofit.zero_grad()
        optimizer_drop.zero_grad()
        #Backward pass: compute gradient of the loss with respect to all the learnable parameters
        loss.backward()
        loss_drop.backward()
        #the step function on an Optimizer makes an update to its parameters
        optimizer_ofit.step()
        optimizer_drop.step()

        print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 0, loss 99.51759338378906
epoch 0, loss 125.62913513183594
epoch 0, loss 121.64593505859375
epoch 0, loss 42.50615692138672
epoch 0, loss 43.980899810791016
epoch 0, loss 36.30455017089844
epoch 0, loss 35.15445327758789
epoch 0, loss 30.719284057617188
epoch 0, loss 20.001373291015625
epoch 1, loss 21.920326232910156
epoch 1, loss 14.798727989196777
epoch 1, loss 13.31440544128418
epoch 1, loss 10.884588241577148
epoch 1, loss 8.104146957397461
epoch 1, loss 9.201061248779297
epoch 1, loss 10.452159881591797
epoch 1, loss 7.347440719604492
epoch 1, loss 7.562754154205322
epoch 2, loss 5.161434173583984
epoch 2, loss 3.7469332218170166
epoch 2, loss 5.581744194030762
epoch 2, loss 4.136096954345703
epoch 2, loss 3.4042301177978516
epoch 2, loss 4.3454389572143555
epoch 2, loss 4.480903625488281
epoch 2, loss 3.8066086769104004
epoch 2, loss 2.840787649154663
epoch 3, loss 2.9470748901367188
epoch 3, loss 2.598339080810547
epoch 3, loss 1.886122226715088
epoch 3, loss 1.9864530563

In [86]:
yhat[0:10]

tensor([[ -7.1870,  11.4946,  13.3781,  20.0181, -11.4802, -26.9094, -10.9743],
        [ -1.7473,  10.6897,   4.1571,  -3.5382,  75.0565, -26.7635,  20.0978],
        [ 12.0374, -14.3592,  -8.3758,  21.1063,  -9.1857, -16.8022,  -0.9614],
        [-13.7234,   5.6359,  20.4721,  14.1230, -12.6644, -23.1356,  -7.5964],
        [  0.3019,  10.5231,   3.6058,  20.7109, -12.7297, -25.6573,  -6.8307],
        [  8.5930, -28.9212, -13.6710,   5.0726,  -4.0138,  -1.6518,  19.5210],
        [  0.4285,  30.6917,  13.9260,  19.5872, -11.4440, -26.3133,  -6.0863],
        [  5.5318, -12.3729,   1.5273,  17.0668, -12.1962, -16.2911,  -4.7175],
        [-13.7234,   5.6359,  20.4721,  14.1230, -12.6644, -23.1356,  -7.5964],
        [ 12.0374, -14.3592,  -8.3758,  21.1063,  -9.1857, -16.8022,  -0.9614]],
       grad_fn=<SliceBackward0>)

In [87]:
torch.max(yhat.data,1)

torch.return_types.max(
values=tensor([20.0181, 75.0565, 21.1063, 20.4721, 20.7109, 19.5210, 30.6917, 17.0668,
        20.4721, 21.1063, 17.0668, 18.2243, 20.4420, 34.2145, 18.2134, 40.3404,
        14.9849, 19.7281, 19.7281, 20.1001, 21.1829, 28.9143, 31.9816, 11.2492,
        14.8805, 14.3560, 22.9516, 19.3964, 24.8851, 18.4756, 11.3676, 20.7084,
        42.3363, 15.7891, 32.3671, 30.6917, 12.6259, 31.0114, 16.7061, 19.4993,
        68.8651, 18.4756, 28.7964, 26.2866, 34.3980, 32.3671, 18.2103, 22.1982,
        75.0565, 18.2103, 71.7950, 14.7793, 14.3560, 28.9143, 19.4978, 19.4993,
        24.8851, 19.8134, 42.3363,  9.1454, 12.6259, 19.8569, 99.0987, 71.7950,
        62.4857, 18.2134, 13.0802, 13.0802,  7.4222, 32.3321, 26.2866, 30.6173,
        20.1020, 18.2243, 31.0114,  8.5145, 32.3321, 62.4857, 40.3404, 68.8651,
         8.5145, 20.7109, 20.1020, 15.7891, 28.7424]),
indices=tensor([3, 4, 3, 2, 3, 6, 1, 3, 2, 3, 3, 3, 6, 1, 3, 0, 2, 3, 3, 3, 3, 6, 1, 5,
        3, 2, 6, 2, 3, 3, 

In [88]:
y

tensor([5, 3, 3, 3, 0])

In [89]:
z = model(x_val)
z_dropout = model_drop(x_val)

In [90]:
_,yhat=torch.max(z.data,1)
yhat[0:20]

tensor([3, 4, 4, 3, 3, 3, 2, 0, 4, 1, 3, 2, 5, 6, 3, 2, 3, 0, 0, 6])

In [91]:
_,yhat_dropout=torch.max(z_dropout.data,1)
yhat_dropout[0:20]

tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [92]:
y_val[0:20]

tensor([3, 4, 4, 3, 3, 3, 2, 4, 4, 1, 3, 2, 5, 6, 3, 2, 3, 4, 0, 6])

In [93]:
# Making the Confusion Matrix
eval_matrix = (pd.crosstab(y_val, yhat))
print(eval_matrix)

col_0  0  1  2   3  4  5  6
row_0                      
0      2  0  0   0  0  0  0
1      0  4  0   0  0  0  0
2      0  0  6   0  0  0  0
3      0  0  0  15  0  0  0
4      2  0  0   0  5  0  0
5      0  0  0   0  0  3  0
6      0  0  0   0  0  0  6


In [94]:
# Making the Confusion Matrix
eval_matrix_dropout = (pd.crosstab(y_val, yhat_dropout))
print(eval_matrix_dropout)

col_0   3
row_0    
0       2
1       4
2       6
3      15
4       7
5       3
6       6


In [95]:
(eval_matrix[0][0]+eval_matrix[1][1]+eval_matrix[2][2])/y_val.shape[0]

0.27906976744186046

In [96]:
(eval_matrix_dropout[0][0]+eval_matrix_dropout[1][1]+eval_matrix_dropout[2][2])/y_val.shape[0]

KeyError: 0